## Let us do some tests related to the efficiency of numpy and scipy functions

In [1]:
import numpy as np

import scipy.sparse

from scipy.sparse import csc_matrix, csr_matrix

### Generate sparse random data

In [2]:
n = 5000
dim = 2000
A = csr_matrix(scipy.sparse.random(n, dim))
x = csr_matrix(scipy.sparse.random(dim, 1))
A_ = csc_matrix(A)
x_ = csc_matrix(x)

### Compare csc_matrix and csr_matrix

In [3]:
%timeit A.dot(x)

322 µs ± 5.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [4]:
%timeit A.dot(x_)

376 µs ± 18.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [5]:
%timeit A_.dot(x)

149 µs ± 1.34 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [6]:
%timeit A_.dot(x_)

96.1 µs ± 2.18 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


#### Let's check that the results match.

In [7]:
abs(A.dot(x) - A.dot(x_)).sum()

0.0

#### Is Numpy faster?

In [8]:
x = x.toarray().squeeze()
A = A.toarray()

In [9]:
%timeit A @ x

4.44 ms ± 101 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### Now let us sample rows (used in SGD)

In [10]:
def fake_stochastic_gradient(A, x):
    n = A.shape[0]
    i = np.random.choice(n)
    return A[i].dot(x)

In [11]:
%timeit fake_stochastic_gradient(A, x)

5.78 µs ± 39 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [12]:
%timeit fake_stochastic_gradient(A, x_)

72.8 ms ± 845 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
%timeit fake_stochastic_gradient(A_, x)

214 µs ± 5.04 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [14]:
%timeit fake_stochastic_gradient(A_, x_)

298 µs ± 2.43 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Conclusions:
### 1. Use csc for deterministic (full batch) gradient computation
### 2. Use csr if stochastic gradients are required

## Efficiencty of other sparse-vector operations

In [32]:
x = csr_matrix([1] + list(np.zeros(1000000)))
x

<1x1000001 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [33]:
%timeit x + x

73.8 µs ± 1.37 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [34]:
%timeit x * 2

33.6 µs ± 1.69 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [35]:
x = csr_matrix(np.arange(1000000))
x

<1x1000000 sparse matrix of type '<class 'numpy.int64'>'
	with 999999 stored elements in Compressed Sparse Row format>

In [36]:
%timeit x + x

3.46 ms ± 270 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [37]:
%timeit x * 2

1.81 ms ± 58.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Random number generation

### Todo